In [1]:
import pandas as pd 
import numpy as np 
import scanpy as sc
import matplotlib.pyplot as plt
import concurrent.futures
import pickle
import warnings
from datetime import date
import hisepy
import os
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed,ProcessPoolExecutor
from tqdm import tqdm
import anndata
import gc
from cellphonedb.src.core.methods import cpdb_analysis_method
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method
warnings.filterwarnings("ignore")
sc.settings.n_jobs = 60
print("Current working directory:", os.getcwd())

Current working directory: /home/jupyter/BRI_Figures_Final_V1/Figure6/01_CellPhoneDB


In [2]:
meta_data=pd.read_csv("/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA_meta_data-2024-05-09.csv")

In [3]:
file_names= ['/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA/BRI/h5ad/sample_h5ad/'+x+".h5ad" for x in meta_data['pbmc_sample_id'].tolist()]

# CM CD4 with Core Memory B cell +effector B cells+naive B

In [15]:
# Initialize combined DataFrames
combined_mean = pd.DataFrame()
combined_pvalues = pd.DataFrame()
combined_significant_means = pd.DataFrame()
combined_deconvoluted = pd.DataFrame()
combined_deconvoluted_percent = pd.DataFrame()

# Function to process each adata
def process_adata(file_path, index):
    adata=sc.read_h5ad(file_path)
    adata_subset = adata[adata.obs["AIFI_L3"].isin(["CM CD4 T cell", "Core memory B cell","CD27+ effector B cell", "CD27- effector B cell","Core naive B cell"])]
    sc.pp.normalize_total(adata_subset, target_sum=1e4)
    sc.pp.log1p(adata_subset)
    adata_subset.write_h5ad(f"tmp/tmp_{index}.h5ad")
    
    meta_data = adata_subset.obs[["barcodes", "AIFI_L3"]]
    meta_data.columns = ["Cell", "cell_type"]
    meta_data.to_csv(f"tmp/meta_data_tmp_{index}.csv", index=False)

    cpdb_results = cpdb_statistical_analysis_method.call(
        cpdb_file_path="v5.0.0/cellphonedb.zip",
        meta_file_path=f"tmp/meta_data_tmp_{index}.csv",
        counts_file_path=f"tmp/tmp_{index}.h5ad",
        counts_data='hgnc_symbol',
        output_suffix=f"{index}",
        output_path='tmp/'
    )

    # Read and append results
    statistical_analysis_mean = pd.read_csv(f"tmp/statistical_analysis_means_{index}.txt", sep="\t")
    statistical_analysis_pvalues = pd.read_csv(f"tmp/statistical_analysis_pvalues_{index}.txt", sep="\t")
    statistical_analysis_significant_means = pd.read_csv(f"tmp/statistical_analysis_significant_means_{index}.txt", sep="\t")
    statistical_analysis_deconvoluted = pd.read_csv(f"tmp/statistical_analysis_deconvoluted_{index}.txt", sep="\t")
    statistical_analysis_deconvoluted_percent = pd.read_csv(f"tmp/statistical_analysis_deconvoluted_percents_{index}.txt", sep="\t")
    
    specimen_id = adata.obs['specimen.specimenGuid'][0]
    statistical_analysis_mean["pbmc_sample_id"] = specimen_id
    statistical_analysis_pvalues["pbmc_sample_id"] = specimen_id
    statistical_analysis_significant_means["pbmc_sample_id"] = specimen_id
    statistical_analysis_deconvoluted["pbmc_sample_id"] = specimen_id
    statistical_analysis_deconvoluted_percent["pbmc_sample_id"] = specimen_id
    
    return (statistical_analysis_mean, statistical_analysis_pvalues, 
            statistical_analysis_significant_means, statistical_analysis_deconvoluted, 
            statistical_analysis_deconvoluted_percent)
os.mkdir('tmp')
# Process the adata_list in parallel
with concurrent.futures.ProcessPoolExecutor(max_workers=60) as executor:
    futures = [executor.submit(process_adata, file_path, i) for i, file_path in enumerate(file_names)]
    with tqdm(total=len(file_names), desc="Processing adata subsets") as pbar:
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            combined_mean = pd.concat([combined_mean, result[0]])
            combined_pvalues = pd.concat([combined_pvalues, result[1]])
            combined_significant_means = pd.concat([combined_significant_means, result[2]])
            combined_deconvoluted = pd.concat([combined_deconvoluted, result[3]])
            combined_deconvoluted_percent = pd.concat([combined_deconvoluted_percent, result[4]])
            pbar.update(1)
shutil.rmtree("tmp")
# Save the combined results
combined_mean.to_csv("combined_statistical_analysis_means.csv", index=False)
combined_pvalues.to_csv("combined_statistical_analysis_pvalues.csv", index=False)
combined_significant_means.to_csv("combined_statistical_analysis_significant_means.csv", index=False)
combined_deconvoluted.to_csv("combined_statistical_analysis_deconvoluted.csv", index=False)
combined_deconvoluted_percent.to_csv("combined_statistical_analysis_deconvoluted_percent.csv", index=False)

Processing adata subsets:   0% 0/868 [00:00<?, ?it/s]

Reading user files...
Reading user files...
Reading user files...
Reading user files...
Reading user files...
Reading user files...
Reading user files...
Reading user files...
Reading user files...
Reading user files...
Reading user files...
Reading user files...Reading user files...

The following user files were loaded successfully:Reading user files...

tmp/tmp_33.h5ad
tmp/meta_data_tmp_33.csv
Reading user files...Reading user files...
Reading user files...
Reading user files...
Reading user files...

Reading user files...Reading user files...
Reading user files...

Reading user files...
Reading user files...
Reading user files...
Reading user files...Reading user files...
Reading user files...

Reading user files...Reading user files...

Reading user files...
Reading user files...
Reading user files...
Reading user files...
Reading user files...
Reading user files...Reading user files...

Reading user files...
Reading user files...
Reading user files...
The following user files wer

  0% 0/1000 [00:00<?, ?it/s]

The following user files were loaded successfully:
tmp/tmp_57.h5ad
tmp/meta_data_tmp_57.csv

[ ][CORE][20/06/24-02:19:18][INFO] [Cluster Statistical Analysis] Threshold:0.1 Iterations:1000 Debug-seed:-1 Threads:4 Precision:3
tmp/meta_data_tmp_40.csv
[ ][CORE][20/06/24-02:19:18][INFO] [Cluster Statistical Analysis] Threshold:0.1 Iterations:1000 Debug-seed:-1 Threads:4 Precision:3
[ ][CORE][20/06/24-02:19:18][INFO] [Cluster Statistical Analysis] Threshold:0.1 Iterations:1000 Debug-seed:-1 Threads:4 Precision:3
[ ][CORE][20/06/24-02:19:18][INFO] [Cluster Statistical Analysis] Threshold:0.1 Iterations:1000 Debug-seed:-1 Threads:4 Precision:3
[ ][CORE][20/06/24-02:19:18][INFO] [Cluster Statistical Analysis] Threshold:0.1 Iterations:1000 Debug-seed:-1 Threads:4 Precision:3
[ ][CORE][20/06/24-02:19:18][INFO] [Cluster Statistical Analysis] Threshold:0.1 Iterations:1000 Debug-seed:-1 Threads:4 Precision:3
The following user files were loaded successfully:The following user files were loaded suc

  0% 0/1000 [00:00<?, ?it/s]

...

100% 1000/1000 [00:48<00:00, 20.79it/s][31:26<00:50,  3.18it/s]


[ ][CORE][20/06/24-02:50:43][INFO] Building Pvalues result
[ ][CORE][20/06/24-02:50:43][INFO] Building results


Processing adata subsets:  82% 708/868 [31:26<00:51,  3.10it/s]

Saved deconvoluted to tmp/statistical_analysis_deconvoluted_867.txt
Saved deconvoluted_percents to tmp/statistical_analysis_deconvoluted_percents_867.txt
Saved means to tmp/statistical_analysis_means_867.txt
Saved pvalues to tmp/statistical_analysis_pvalues_867.txt
Saved significant_means to tmp/statistical_analysis_significant_means_867.txt


Processing adata subsets: 100% 868/868 [32:22<00:00,  2.24s/it]
